# Basic liquid handling

In this notebook, you will learn how to use PyHamilton to move water from one range of wells to another.

**Note: before running this notebook, you should have**:

- Installed PyHamilton and the USB driver as described in [the installation guide](installation).
- Connected the Hamilton to your computer using the USB cable.

Video of what this code does:

<iframe width="640" height="360" src="https://www.youtube.com/embed/NN6ltrRj3bU" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

## Setting up a connection with the robot

Start by importing the {class}`~pylabrobot.liquid_handling.liquid_handler.LiquidHandler` class, which will serve as a front end for all liquid handling operations.

Backends serve as communicators between between `LiquidHandler`s and the actual hardware. Since we are using a Hamilton STAR, we also import the {class}`~pylabrobot.liquid_handling.backends.STAR` backend.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pylabrobot.liquid_handling import LiquidHandler

from pylabrobot.liquid_handling.backends import STAR
from pylabrobot.liquid_handling.backends.chatterbox_backend import ChatterBoxBackend
from pylabrobot.liquid_handling.backends.saver_backend import SaverBackend

In addition, import the {class}`~pylabrobot.resources.hamilton.STARLetDeck`, which represents the deck of the Hamilton STAR.

In [ ]:
from pylabrobot.resources.hamilton import STARLetDeck

Create a new liquid handler using `STAR` as its backend.

In [ ]:
# backend = STAR()
backend = ChatterBoxBackend(num_channels=1)
#backend = SaverBackend(num_channels=1)

lh = LiquidHandler(backend=backend, deck=STARLetDeck())

The final step is to open communication with the robot. This is done using the {func}`~pylabrobot.liquid_handling.LiquidHandler.setup` method.

In [ ]:
await lh.setup()

## Creating the deck layout

Now that we have a `LiquidHandler` instance, we can define the deck layout.

The layout in this tutorial will contain five sets of standard volume tips with filter, 1 set of 96 1mL wells, and tip and plate carriers on which these resources are positioned.

Start by importing the relevant objects and variables from the PyHamilton package. This notebook uses the following resources:

- {class}`~pylabrobot.resources.ml_star.tip_carriers.TIP_CAR_480_A00` tip carrier
- {class}`~pylabrobot.resources.ml_star.plate_carriers.PLT_CAR_L5AC_A00` plate carrier
- {class}`~pylabrobot.resources.corning_costar.plates.Cos_96_DW_1mL` wells
- {class}`~pylabrobot.resources.ml_star.tip_racks.HTF_L` tips

In [ ]:
from pylabrobot.resources import (
    TIP_CAR_480_A00,
    PLT_CAR_L5AC_A00,
    Cos_96_DW_1mL,
    HTF_L
)

Then create a tip carrier named `tip carrier`, which will contain tip rack at all 5 positions. These positions can be accessed using `tip_car[x]`, and are 0 indexed.

In [ ]:
tip_car = TIP_CAR_480_A00(name='tip carrier')
tip_car[0] = HTF_L(name='tips_01', with_tips=True)

Use {func}`~pylabrobot.resources.abstract.assign_child_resources` to assign the tip carrier to the deck of the liquid handler. All resources contained by this carrier will be assigned automatically.

In the `rails` parameter, we can pass the location of the tip carrier. The locations of the tips will automatically be calculated.

In [ ]:
lh.deck.assign_child_resource(tip_car, rails=3)

Repeat this for the plates.

In [ ]:
plt_car = PLT_CAR_L5AC_A00(name='plate carrier')
plt_car[0] = Cos_96_DW_1mL(name='plate_01')

In [ ]:
lh.deck.assign_child_resource(plt_car, rails=15)

Let's look at a summary of the deck layout using {func}`~pylabrobot.liquid_handling.LiquidHandler.summary`.

In [ ]:
lh.summary()

## Picking up tips

Picking up tips is as easy as querying the tips from the tiprack.

In [ ]:
tiprack = lh.deck.get_resource("tips_01")
await lh.pick_up_tips(tiprack["A1:C1"])

## Aspirating and dispensing

Aspirating and dispensing work similarly to picking up tips: where you use booleans to specify which tips to pick up, with aspiration and dispensing you use floats to specify the volume to aspirate or dispense in $\mu L$.

The cells below move liquid from wells `'A1:C1'` to `'D1:F1'` using channels 1, 2, and 3 using the {func}`~pylabrobot.liquid_handling.LiquidHandler.aspirate` and {func}`~pylabrobot.liquid_handling.LiquidHandler.dispense` methods.

In [ ]:
plate = lh.deck.get_resource("plate_01")
await lh.aspirate(plate["A1:C1"], vols=[100.0, 50.0, 200.0])

After the liquid has been aspirated, dispense it in the wells below. Note that while we specify different wells, we are still using the same channels. This is needed because only these channels contain liquid, of course.

In [ ]:
await lh.dispense(plate["D1:F1"], vols=[100.0, 50.0, 200.0])

Let's move the liquid back to the original wells.

In [ ]:
await lh.aspirate(plate["D1:F1"], vols=[100.0, 50.0, 200.0])
await lh.dispense(plate["A1:C1"], vols=[100.0, 50.0, 200.0])

## Discarding tips

Finally, you can discard tips by using the {func}`~pylabrobot.liquid_handling.LiquidHandler.discard_tips` method.

In [ ]:
await lh.drop_tips(tiprack["A1:C1"])

In [ ]:
await lh.stop()